# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display



In [2]:
# constants
load_dotenv(override=True)

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
MODEL_NAME = "openai/gpt-4o-mini"
SYSTEM_PROMPT = (
    "You are a helpful assistant that explains technical questions "
    "in a simple and easy to understand way."
)


In [3]:
# OpenRouter setup (fail fast if missing)
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not set. Add it to .env (https://openrouter.ai/keys)")

print(f"OpenRouter API key loaded (starts with {OPENROUTER_API_KEY[:8]}...)")
client = OpenAI(api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_BASE_URL)



OpenRouter API key loaded (starts with sk-or-v1...)


In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# Stream explanation and update one Markdown output in place

def stream_explanation(question: str, client: OpenAI, system_prompt: str = SYSTEM_PROMPT) -> str:
    """Return full response while streaming chunks to notebook output."""
    stream = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
        ],
        stream=True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        delta = chunk.choices[0].delta.content if chunk.choices else None
        if delta:
            response += delta
            display_handle.update(Markdown(response))
    return response


answer = stream_explanation(question, client=client)

This code is using Python and involves a couple of concepts: **set comprehensions** and the `yield from` statement. Let's break it down step by step.

1. **Understanding the Context**:
   - Assume that `books` is a collection (like a list) of dictionaries, where each dictionary represents a book with various attributes, including an "author".

2. **Set Comprehension**: 
   - `{book.get("author") for book in books if book.get("author")}`: 
     - This part of the code is creating a **set** (which is a collection that automatically removes duplicates) of the authors of the books.
     - `book.get("author")` tries to get the "author" value from each `book` dictionary.
     - The `if book.get("author")` part ensures that we only include the authors that actually exist (i.e., are not `None` or empty).

3. **Yield from**:
   - `yield from` is used in Python generators to yield all values from an iterable (in this case, our set of authors) one by one. When this line is executed, it produces each author in the set, allowing the caller of the generator to receive each author value individually.

So, putting it all together:

- This line of code takes a collection of book dictionaries, extracts the authors (where they exist), removes any duplicates (because it's a set), and then yields each author one at a time. 

### Why Use This Code?
- It's a concise way to gather unique authors from a list of books and make them available for iteration.
- The use of `yield from` suggests that this code is likely part of a generator function, allowing for memory-efficient handling of potentially large sets of book records because you don't need to store all the authors in a list; they're produced one at a time as needed. 

### Example:
If `books` were defined like this:
```python
books = [
    {"title": "Book One", "author": "Alice"},
    {"title": "Book Two", "author": "Bob"},
    {"title": "Book Three", "author": "Alice"},
    {"title": "Book Four", "author": None},
]
```
The code would yield the authors.
- The resulting unique author set from this would be `{"Alice", "Bob"}`, and you could iterate over them one at a time.